# PART 1: BUILD A RAG USING LANGCHAIN AND OPENAI

In [14]:
%pip install python-dotenv
%pip install openai==0.28
%pip install langchain
%pip install -U langchain-core
%pip install pinecone


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pinecone from https://files.pythonhosted.org/packages/2a/16/169e969df341a32f6d1d9ace6f658b2d0ace3af11ae9294e29925c8c5f09/pinecone-5.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for pinecone-plugin-inference<4.0.0,>=2.0.0 from

## Prueba de conexión básica con OpenAI

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

try:
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Test API key"}],
        max_tokens=5
    )
    print("Respuesta:", response['choices'][0]['message']['content'].strip())
except openai.error.AuthenticationError:
    print("Error: La API Key no es válida o no está configurada correctamente.")
except openai.error.OpenAIError as e:
    print(f"Error al conectar con la API: {e}")

Respuesta: I'm sorry, but


## Prueba de integración LangChain con OpenAI para generar respuestas basadas en una base de conocimientos

In [11]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate

# Cargar variables de entorno
load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(
    model="gpt-4",
    openai_api_key=api_key
)

# Inicializar embeddings de OpenAI
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# Cargar y dividir el contenido
loader = WebBaseLoader("https://www.escuelaing.edu.co/es/programas/maestria-en-informatica/")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

/var/folders/ql/j31d4jwd3n9bq05wmv6whz140000gn/T/ipykernel_61251/51267402.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/ql/j31d4jwd3n9bq05wmv6whz140000gn/T/ipykernel_61251/51267402.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=api_key)


In [12]:
# Using InMemoryVectorStore

# Extraer texto de los documentos divididos
document_texts = [doc.page_content for doc in all_splits]

# Crear el vector store en memoria y cargar documentos
vector_store = InMemoryVectorStore.from_texts(
    texts=document_texts,
    embedding=embeddings
)

# Definir la plantilla del prompt
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Usa el siguiente contexto para responder la pregunta:\n"
        "{context}\n\n"
        "Pregunta: {question}\n"
        "Respuesta:"
    )
)

# Función para recuperar contexto y generar respuesta
def retrieve_and_answer(question):
    # Buscar documentos relevantes
    retrieved_docs = vector_store.similarity_search(question, k=5)
    
    # Extraer el contenido de cada documento
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # Formatear el prompt
    prompt = prompt_template.format(context=context, question=question)
    
    # Generar respuesta
    response = llm.invoke(prompt)
    return response

# Probar la implementación
question = "¿Por qué estudiar Maestría en Informática?"
answer = retrieve_and_answer(question)
print("Pregunta:", question)
print("Respuesta:", answer)

Pregunta: ¿Por qué estudiar Maestría en Informática?
Respuesta: content='La Maestría en Informática de la Universidad Escuela Colombiana de Ingeniería Julio Garavito ofrece una oportunidad única para profundizar en los conocimientos computacionales y en la infraestructura para procesar y analizar datos. Este programa responde a las crecientes necesidades técnicas relacionadas con el uso eficiente de las Tecnologías de la Información y la Comunicación (TIC) y la protección de los datos. Entre las principales razones para estudiar esta maestría se encuentran:\n\n1. Crecimiento exponencial de los datos: Los datos están creciendo a un ritmo acelerado y necesitan ser procesados eficientemente. Esta maestría proporciona las habilidades necesarias para manejar grandes volúmenes de datos y extraer información útil de ellos.\n\n2. Alta demanda de software eficiente y de calidad: La maestría enseña cómo desarrollar software que sea eficiente, efectivo y de alta calidad. Los graduados estarán bie

In [19]:
# Using FAISS

# Create a FAISS VectorStore for indexing
vector_store = FAISS.from_documents(all_splits, embeddings)

# Define a question-answering prompt
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Use the following context to answer the question:\n"
        "{context}\n\n"
        "Question: {question}\n"
        "Answer:"
    )
)

def retrieve_and_answer(question):
    retrieved_docs = vector_store.similarity_search(question, k=5)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    prompt = prompt_template.format(context=context, question=question)
    
    # Generate response
    response = llm.invoke(prompt)
    return response

# Test the implementation
question = "Por qué estudir Maestría en Informática?"
answer = retrieve_and_answer(question)
print("Question:", question)
print("Answer:", answer)


Question: Por qué estudir Maestría en Informática?
Answer: content='Estudiar la Maestría en Informática en la Universidad Escuela Colombiana de Ingeniería Julio Garavito tiene varios beneficios. Primero, responde a las necesidades técnicas actuales relacionadas con el uso eficiente de las TIC y la protección de los datos. En este sentido, prepara a los estudiantes para enfrentar el crecimiento exponencial de los datos y la alta demanda de software eficiente y de calidad. Además, aborda la aparición y aumento de amenazas cibernéticas que afectan a personas, organizaciones y gobiernos en el mundo. \n\nPor otro lado, la universidad tiene una reconocida experiencia de 30 años en ingeniería de sistemas y áreas afines, y hace aproximadamente 10 años que se investiga en el área de ciberseguridad, ingeniería de software, análisis de datos, big data e inteligencia de negocios. \n\nFinalmente, el programa forma a los estudiantes como agentes de cambio, profundizando en los conocimientos computac

## Análisis de Arquitectura del RAG

## Descripción General
La arquitectura tiene 4 partes:

1. **Carga y preprocesamiento de datos:** Se cargan documentos desde una fuente web y se dividen en fragmentos manejables.
2. **Embeddings e indexación:** Los fragmentos de texto se transforman en embeddings (representaciones vectoriales) y se almacenan en InMemoryVectorStore para búsquedas eficientes.
3. **Procesamiento de consultas:** Las consultas del usuario se convierten en embeddings y se usan para recuperar los fragmentos más relevantes.
4. **Generación de respuestas:** Una plantilla de prompt combina el contexto recuperado con la consulta del usuario para generar una respuesta utilizando GPT-4.



## Flujo Completo de la Consulta

- Consulta del Usuario: El usuario pregunta: "¿Por qué estudiar Maestría en Informática?"
- Embedding de la Consulta: La consulta se convierte en un vector mediante OpenAIEmbeddings.
- Recuperación: Se busca el embedding de la consulta en InMemoryVectorStore para identificar los 5 fragmentos más relevantes.
- Construcción del Prompt: 
    - Los fragmentos recuperados se combinan en un contexto.
    - El contexto y la consulta se insertan en la plantilla del prompt.
- Generación de Respuesta: El prompt se envía a GPT-4, que genera una respuesta basada en el contexto proporcionado.
- Resultado: La respuesta se muestra al usuario.


## Consideraciones

**InMemoryVectorStore** almacena todo en memoria, lo que lo hace inadecuado para conjuntos de datos muy grandes.

# PART 2: CONNECT THE RAG TO PINECONE

In [21]:
%pip install pinecone


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as LangChainPinecone

pinecode_api_key = os.environ["PINECONE_API_KEY"]
pc = Pinecone(api_key=pinecode_api_key)
pinecone_index_name = "langchain-rag-aygo"

pinecone_index = pc.Index(pinecone_index_name)

# Crear el vector store con Pinecone y agregar documentos
vector_store = LangChainPinecone.from_documents(
    documents=all_splits,
    embedding=embeddings,
    index_name=pinecone_index_name
)

# Definir la plantilla del prompt
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Usa el siguiente contexto para responder la pregunta:\n"
        "{context}\n\n"
        "Pregunta: {question}\n"
        "Respuesta:"
    )
)

# Función para recuperar contexto y generar respuesta
def retrieve_and_answer(question):

    retrieved_docs = vector_store.similarity_search(question, k=5)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    prompt = prompt_template.format(context=context, question=question)
    response = llm.invoke(prompt)
    
    return response

# Probar la implementación
question = "¿Por qué estudiar Maestría en Informática?"
answer = retrieve_and_answer(question)
print("Pregunta:", question)
print("Respuesta:", answer)


Pregunta: ¿Por qué estudiar Maestría en Informática?
Respuesta: content='Estudiar una Maestría en Informática puede proporcionar una serie de beneficios. Primero, puede ayudar a profundizar tus conocimientos en áreas específicas de la informática, como la inteligencia artificial, la ciberseguridad, la ciencia de datos, entre otras. Esto puede abrirte la puerta a roles de trabajo más especializados y potencialmente más lucrativos. Además, un título de maestría puede hacer que te destaques en un campo competitivo, lo que podría aumentar tus posibilidades de conseguir un trabajo. También puede proporcionarte las habilidades necesarias para realizar investigaciones en el campo de la informática, lo cual es particularmente útil si estás interesado en un doctorado o una carrera en la academia.' additional_kwargs={} response_metadata={'token_usage': <OpenAIObject at 0x1279664e0> JSON: {
  "prompt_tokens": 35,
  "completion_tokens": 162,
  "total_tokens": 197,
  "prompt_tokens_details": {
    

## Explicación de los Cambios

- **Uso de Pinecone como Almacenamiento Vectorial:**
    - Los documentos se almacenan en Pinecone, que permite búsquedas rápidas utilizando vectores de embeddings.
    - Reemplaza el uso de almacenamiento en memoria, haciéndolo más escalable y persistente.
- **Conexión a un Índice Existente:**
    - Se utiliza Pinecone.Index para conectar al índice de Pinecone previamente creado.
    - Esto asegura que los documentos se guarden y se recuperen en un entorno persistente.
- **Integración con LangChain:**
    - LangChainPinecone.from_documents utiliza los embeddings para almacenar documentos en Pinecone.
    - Se proporciona el índice existente y los embeddings generados.
- **Búsqueda de Similitud:** similarity_search realiza búsquedas eficientes en Pinecone, recuperando documentos relevantes para una consulta dada.

## Comparaciones

| **Aspecto**                | **InMemoryVectorStore**                           | **Pinecone**                                       |
|----------------------------|--------------------------------------------------|--------------------------------------------------|
| **Persistencia**            | No, los datos se pierden al finalizar el programa. | Sí, los datos permanecen guardados en el índice. |
| **Escalabilidad**           | Limitado a la memoria del sistema.                | Alta, maneja grandes volúmenes de datos.         |
| **Velocidad de Búsqueda**   | Muy rápida para conjuntos pequeños (en memoria).  | Rápida para conjuntos grandes (basado en red).   |
| **Tiempo de Configuración** | Bajo, no requiere configuración adicional.        | Moderado, necesita inicialización y configuración del índice. |
| **Dependencia de Red**      | No, funciona localmente.                          | Sí, requiere conexión a Pinecone.               |
| **Latencia Promedio**       | ~0.01 - 0.05 segundos (para 1,000 documentos).     | ~0.2 - 0.5 segundos (para 1,000 documentos).     |
| **Capacidad de Consulta**   | Limitada a la memoria y rendimiento local.         | Escalable, permite consultas concurrentes.       |

## Consideraciones

- **Latencia Promedio:**
    - InMemoryVectorStore tiene ventaja para conjuntos pequeños debido a la naturaleza local de la memoria.
    - Pinecone tiene una ligera latencia debido a la consulta a la red, pero se mantiene eficiente para grandes volúmenes.
- **Escalabilidad:**
    - Pinecone es ideal para aplicaciones que manejan miles o millones de documentos.
    - InMemoryVectorStore se limita por la memoria del sistema y no escala bien.
- **Uso Recomendado:**
    - InMemoryVectorStore: Útil para prototipos o pruebas locales con pequeños conjuntos de datos.
    - Pinecone: Ideal para producción, manejo de datos persistentes y aplicaciones de alto rendimiento.